In [ ]:
# notebooks/6_api_inference.ipynb
"""
API Inference Notebook
Example requests, response format, and integration tests for deployed model.
"""

# %% [markdown]
# # API Inference Testing Notebook
# 
# This notebook demonstrates how to:
# 1. Test the Credit Risk Scoring API
# 2. Make predictions with different feature sets
# 3. Validate response formats
# 4. Run integration tests
# 5. Generate test reports

# %% [markdown]
# ## 1. Setup and Configuration

# %%
import requests
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# %%
# API Configuration
API_BASE_URL = "http://localhost:8001"  # Change if your API runs on different port
HEALTH_ENDPOINT = f"{API_BASE_URL}/health"
PREDICT_ENDPOINT = f"{API_BASE_URL}/predict"
MODEL_INFO_ENDPOINT = f"{API_BASE_URL}/model-info"

# Test data configuration
TEST_CASES_FILE = "../tests/test_cases.json"
SAMPLE_PAYLOADS_FILE = "../payloads/sample_payloads.json"

# Create directories if they don't exist
import os
os.makedirs("../payloads", exist_ok=True)
os.makedirs("../tests", exist_ok=True)

# %% [markdown]
# ## 2. Utility Functions

# %%
def print_json(data: Dict, title: str = None):
    """Pretty print JSON data"""
    if title:
        print(f"\n{'='*60}")
        print(f"{title}")
        print(f"{'='*60}")
    print(json.dumps(data, indent=2))

def test_health_endpoint():
    """Test API health endpoint"""
    try:
        response = requests.get(HEALTH_ENDPOINT, timeout=10)
        if response.status_code == 200:
            data = response.json()
            print_json(data, "Health Check Response")
            
            # Check model status
            if data.get("model_loaded"):
                print("✅ Model is loaded and ready")
                return True
            else:
                print("❌ Model not loaded")
                return False
        else:
            print(f"❌ Health check failed: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print(f"❌ Cannot connect to API at {API_BASE_URL}")
        print("Make sure the API is running:")
        print("  docker-compose up -d")
        return False
    except Exception as e:
        print(f"❌ Health check error: {e}")
        return False

def get_model_info():
    """Get information about the loaded model"""
    try:
        response = requests.get(MODEL_INFO_ENDPOINT, timeout=10)
        if response.status_code == 200:
            data = response.json()
            print_json(data, "Model Information")
            
            # Extract important info
            features = data.get("features_required", [])
            if features:
                print(f"\n📋 Model expects {len(features)} features:")
                for i, feature in enumerate(features, 1):
                    print(f"  {i:2d}. {feature}")
            
            thresholds = data.get("risk_thresholds", {})
            if thresholds:
                print(f"\n⚖️  Risk Thresholds:")
                for category, threshold in thresholds.items():
                    print(f"  - {category}: < {threshold}")
            
            return data
        else:
            print(f"❌ Model info failed: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Model info error: {e}")
        return None

# %% [markdown]
# ## 3. Initial API Health Check

# %%
# Test API connection
print("Testing API connection...")
api_healthy = test_health_endpoint()

if api_healthy:
    # Get model information
    model_info = get_model_info()
else:
    print("\n⚠️  API not healthy. Please start the API first:")
    print("  docker-compose -f docker_compose_2.yml up -d")
    print("\nThen run the next cell to continue...")

# %% [markdown]
# ## 4. Create Sample Test Payloads

# %%
# Create sample payloads for testing
sample_payloads = {
    "simple_test": {
        "customer_id": "CUST_TEST_001",
        "features": {
            "Year_mean": 2024.0,
            "Month_mean": 6.0
        }
    },
    "low_risk_example": {
        "customer_id": "CUST_LOW_001",
        "features": {
            "Year_mean": 2023.0,
            "Month_mean": 3.0
        }
    },
    "medium_risk_example": {
        "customer_id": "CUST_MED_001",
        "features": {
            "Year_mean": 2024.0,
            "Month_mean": 8.0
        }
    },
    "high_risk_example": {
        "customer_id": "CUST_HIGH_001",
        "features": {
            "Year_mean": 2022.0,
            "Month_mean": 11.0
        }
    },
    "edge_case_min": {
        "customer_id": "EDGE_MIN",
        "features": {
            "Year_mean": 2020.0,
            "Month_mean": 1.0
        }
    },
    "edge_case_max": {
        "customer_id": "EDGE_MAX",
        "features": {
            "Year_mean": 2024.0,
            "Month_mean": 12.0
        }
    }
}

# Save sample payloads
with open(SAMPLE_PAYLOADS_FILE, 'w') as f:
    json.dump(sample_payloads, f, indent=2)

print(f"✅ Created {len(sample_payloads)} sample payloads in {SAMPLE_PAYLOADS_FILE}")

# Display sample payloads
print("\n📋 Sample Payloads:")
for name, payload in sample_payloads.items():
    print(f"\n{name}:")
    print(f"  Customer: {payload['customer_id']}")
    print(f"  Features: {payload['features']}")

# %% [markdown]
# ## 5. Test Predictions with Different Payloads

# %%
def make_prediction(payload: Dict) -> Dict:
    """Make a prediction using the API"""
    try:
        start_time = time.time()
        
        response = requests.post(
            PREDICT_ENDPOINT,
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=10
        )
        
        response_time = time.time() - start_time
        
        if response.status_code == 200:
            data = response.json()
            data["response_time_ms"] = round(response_time * 1000, 2)
            data["status_code"] = response.status_code
            return data
        else:
            return {
                "error": f"API Error: {response.status_code}",
                "detail": response.text,
                "response_time_ms": round(response_time * 1000, 2),
                "status_code": response.status_code
            }
            
    except Exception as e:
        return {
            "error": f"Request failed: {str(e)}",
            "response_time_ms": None,
            "status_code": None
        }

# %%
# Test all sample payloads
print("Testing predictions with sample payloads...")
print("="*60)

all_results = {}

for name, payload in sample_payloads.items():
    print(f"\n🔍 Testing: {name}")
    print(f"  Customer: {payload['customer_id']}")
    
    result = make_prediction(payload)
    
    if "error" in result:
        print(f"  ❌ Error: {result['error']}")
    else:
        print(f"  ✅ Success!")
        print(f"  ⏱️  Response time: {result.get('response_time_ms', 'N/A')}ms")
        print(f"  📊 Probability: {result.get('probability', 'N/A'):.4f}")
        print(f"  🎯 Class: {result.get('predicted_class', 'N/A')}")
        print(f"  ⚠️  Risk: {result.get('risk_category', 'N/A')}")
        print(f"  💡 Recommendation: {result.get('recommendation', 'N/A')}")
        
        # Store for analysis
        all_results[name] = result

# %% [markdown]
# ## 6. Response Format Validation

# %%
# Define expected response schema
EXPECTED_RESPONSE_SCHEMA = {
    "probability": float,
    "predicted_class": int,
    "risk_category": str,
    "risk_score": int,
    "recommendation": str,
    "customer_id": str,
    "model": str,
    "timestamp": str,
    "features_used": list
}

def validate_response_format(response: Dict) -> List[str]:
    """Validate response against expected schema"""
    errors = []
    
    for field, expected_type in EXPECTED_RESPONSE_SCHEMA.items():
        if field not in response:
            errors.append(f"Missing required field: {field}")
        elif not isinstance(response[field], expected_type):
            errors.append(f"Field '{field}' has type {type(response[field])}, expected {expected_type}")
    
    # Additional validation rules
    if "probability" in response:
        prob = response["probability"]
        if not (0 <= prob <= 1):
            errors.append(f"Probability {prob} not in range [0, 1]")
    
    if "predicted_class" in response:
        pred_class = response["predicted_class"]
        if pred_class not in [0, 1]:
            errors.append(f"Predicted class {pred_class} not in [0, 1]")
    
    if "risk_score" in response:
        score = response["risk_score"]
        if not (0 <= score <= 100):
            errors.append(f"Risk score {score} not in range [0, 100]")
    
    return errors

# %%
# Validate all successful responses
print("\n🔬 Validating Response Formats...")
print("="*60)

validation_results = {}

for name, result in all_results.items():
    if "error" not in result:
        errors = validate_response_format(result)
        validation_results[name] = {
            "valid": len(errors) == 0,
            "errors": errors
        }
        
        if errors:
            print(f"\n❌ {name} - FAILED:")
            for error in errors:
                print(f"  - {error}")
        else:
            print(f"✅ {name} - PASSED")

# %% [markdown]
# ## 7. Performance Testing

# %%
def performance_test(num_requests: int = 10, payload: Dict = None):
    """Run performance test with multiple requests"""
    if payload is None:
        payload = sample_payloads["simple_test"]
    
    print(f"Running performance test with {num_requests} requests...")
    
    response_times = []
    status_codes = []
    
    for i in range(num_requests):
        print(f"  Request {i+1}/{num_requests}...", end="\r")
        
        start_time = time.time()
        response = requests.post(
            PREDICT_ENDPOINT,
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=10
        )
        response_time = time.time() - start_time
        
        response_times.append(response_time)
        status_codes.append(response.status_code)
    
    print("\n" + "="*60)
    print("📊 Performance Results:")
    print(f"  Total requests: {num_requests}")
    print(f"  Successful: {status_codes.count(200)}")
    print(f"  Failed: {len(status_codes) - status_codes.count(200)}")
    print(f"  Avg response time: {np.mean(response_times)*1000:.2f}ms")
    print(f"  Min response time: {np.min(response_times)*1000:.2f}ms")
    print(f"  Max response time: {np.max(response_times)*1000:.2f}ms")
    print(f"  Std dev: {np.std(response_times)*1000:.2f}ms")
    
    return response_times

# %%
# Run performance test
if api_healthy:
    response_times = performance_test(num_requests=20)
    
    # Plot response time distribution
    plt.figure(figsize=(10, 6))
    plt.hist([rt * 1000 for rt in response_times], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
    plt.axvline(np.mean(response_times)*1000, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(response_times)*1000:.2f}ms')
    plt.xlabel('Response Time (ms)')
    plt.ylabel('Frequency')
    plt.title('API Response Time Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# %% [markdown]
# ## 8. Error Case Testing

# %%
# Test error cases
error_test_cases = {
    "missing_features": {
        "customer_id": "ERROR_001",
        "features": {}  # Empty features
    },
    "wrong_feature_type": {
        "customer_id": "ERROR_002",
        "features": {
            "Year_mean": "2024",  # String instead of number
            "Month_mean": 6.0
        }
    },
    "extra_features": {
        "customer_id": "ERROR_003",
        "features": {
            "Year_mean": 2024.0,
            "Month_mean": 6.0,
            "Extra_Feature": 100.0  # Extra feature not expected by model
        }
    },
    "malformed_json": "This is not JSON",  # Invalid JSON
    "null_values": {
        "customer_id": "ERROR_004",
        "features": {
            "Year_mean": None,  # Null value
            "Month_mean": 6.0
        }
    }
}

# %%
print("\n🧪 Testing Error Cases...")
print("="*60)

error_results = {}

for name, payload in error_test_cases.items():
    print(f"\nTesting: {name}")
    
    try:
        if name == "malformed_json":
            # Send raw string instead of JSON
            response = requests.post(
                PREDICT_ENDPOINT,
                data=payload,
                headers={"Content-Type": "application/json"},
                timeout=10
            )
        else:
            response = requests.post(
                PREDICT_ENDPOINT,
                json=payload,
                headers={"Content-Type": "application/json"},
                timeout=10
            )
        
        error_results[name] = {
            "status_code": response.status_code,
            "response": response.json() if response.status_code != 200 else response.text[:100]
        }
        
        print(f"  Status: {response.status_code}")
        
        if response.status_code >= 400:
            print(f"  ✅ Expected error (status {response.status_code})")
            if response.status_code == 400:
                print(f"  Detail: {response.json().get('detail', 'No detail')}")
        else:
            print(f"  ⚠️  Unexpected success")
            
    except Exception as e:
        error_results[name] = {"error": str(e)}
        print(f"  ❌ Exception: {e}")

# %% [markdown]
# ## 9. Create Test Report

# %%
def generate_test_report():
    """Generate comprehensive test report"""
    report = {
        "timestamp": datetime.now().isoformat(),
        "api_base_url": API_BASE_URL,
        "health_check": "PASSED" if api_healthy else "FAILED",
        "test_summary": {
            "total_test_cases": len(sample_payloads),
            "successful_predictions": sum(1 for r in all_results.values() if "error" not in r),
            "failed_predictions": sum(1 for r in all_results.values() if "error" in r),
            "format_validation_passed": sum(1 for v in validation_results.values() if v["valid"]),
            "format_validation_failed": sum(1 for v in validation_results.values() if not v["valid"])
        },
        "model_info": model_info if 'model_info' in locals() else None,
        "sample_results": {},
        "performance_metrics": {},
        "error_cases": error_results
    }
    
    # Add sample results
    for name, result in all_results.items():
        if "error" not in result:
            report["sample_results"][name] = {
                "probability": result.get("probability"),
                "predicted_class": result.get("predicted_class"),
                "risk_category": result.get("risk_category"),
                "response_time_ms": result.get("response_time_ms")
            }
    
    # Add performance metrics if available
    if 'response_times' in locals():
        report["performance_metrics"] = {
            "avg_response_time_ms": np.mean(response_times) * 1000,
            "min_response_time_ms": np.min(response_times) * 1000,
            "max_response_time_ms": np.max(response_times) * 1000,
            "std_dev_ms": np.std(response_times) * 1000
        }
    
    return report

# %%
# Generate and save test report
test_report = generate_test_report()

report_file = f"../reports/api_test_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
os.makedirs("../reports", exist_ok=True)

with open(report_file, 'w') as f:
    json.dump(test_report, f, indent=2)

print(f"\n📋 Test report saved to: {report_file}")

# Display summary
print("\n" + "="*60)
print("🎉 TEST SUMMARY")
print("="*60)
print(f"API Health: {'✅ PASSED' if test_report['health_check'] == 'PASSED' else '❌ FAILED'}")
print(f"Successful Predictions: {test_report['test_summary']['successful_predictions']}/{test_report['test_summary']['total_test_cases']}")
print(f"Format Validation: {test_report['test_summary']['format_validation_passed']} passed, {test_report['test_summary']['format_validation_failed']} failed")

if test_report['performance_metrics']:
    print(f"Avg Response Time: {test_report['performance_metrics']['avg_response_time_ms']:.2f}ms")

# %% [markdown]
# ## 10. Integration Test Suite

# %%
# Create a comprehensive integration test function
def run_integration_test_suite():
    """Run complete integration test suite"""
    print("🚀 Running Integration Test Suite")
    print("="*60)
    
    test_results = []
    
    # Test 1: API Health
    print("\n1️⃣  Testing API Health...")
    health_ok = test_health_endpoint()
    test_results.append(("API Health", health_ok))
    
    # Test 2: Model Info
    print("\n2️⃣  Testing Model Info...")
    if health_ok:
        info = get_model_info()
        test_results.append(("Model Info", info is not None))
    
    # Test 3: Basic Prediction
    print("\n3️⃣  Testing Basic Prediction...")
    if health_ok:
        payload = sample_payloads["simple_test"]
        result = make_prediction(payload)
        basic_pred_ok = "error" not in result
        test_results.append(("Basic Prediction", basic_pred_ok))
        
        if basic_pred_ok:
            print(f"   Probability: {result.get('probability', 'N/A'):.4f}")
            print(f"   Risk Category: {result.get('risk_category', 'N/A')}")
    
    # Test 4: Response Format
    print("\n4️⃣  Testing Response Format...")
    if health_ok and basic_pred_ok:
        errors = validate_response_format(result)
        format_ok = len(errors) == 0
        test_results.append(("Response Format", format_ok))
        
        if errors:
            print("   Format Errors:")
            for error in errors:
                print(f"   - {error}")
    
    # Test 5: Performance
    print("\n5️⃣  Testing Performance (5 requests)...")
    if health_ok:
        try:
            perf_times = performance_test(num_requests=5)
            perf_ok = all(t < 1.0 for t in perf_times)  # All under 1 second
            test_results.append(("Performance", perf_ok))
        except:
            test_results.append(("Performance", False))
    
    # Test 6: Error Handling
    print("\n6️⃣  Testing Error Handling...")
    error_payload = {"customer_id": "TEST", "features": {}}  # Missing features
    error_result = make_prediction(error_payload)
    error_ok = "error" in error_result or error_result.get("status_code", 200) >= 400
    test_results.append(("Error Handling", error_ok))
    
    # Summary
    print("\n" + "="*60)
    print("📊 INTEGRATION TEST RESULTS")
    print("="*60)
    
    passed = sum(1 for name, ok in test_results if ok)
    total = len(test_results)
    
    for i, (name, ok) in enumerate(test_results, 1):
        status = "✅ PASS" if ok else "❌ FAIL"
        print(f"{i}. {name}: {status}")
    
    print(f"\nOverall: {passed}/{total} tests passed ({passed/total*100:.1f}%)")
    
    return passed == total

# %%
# Run the integration test suite
all_tests_passed = run_integration_test_suite()

if all_tests_passed:
    print("\n🎉 All integration tests passed! API is ready for production use.")
else:
    print("\n⚠️  Some integration tests failed. Please check the API implementation.")

# %% [markdown]
# ## 11. Example: Batch Prediction Script

# %%
# Example batch prediction script
def batch_predict(csv_file_path: str, output_file: str = None):
    """
    Make predictions for multiple customers from a CSV file.
    
    CSV should have columns matching feature names, plus optional 'customer_id'
    """
    try:
        # Read input data
        df = pd.read_csv(csv_file_path)
        
        print(f"Processing {len(df)} records...")
        
        results = []
        
        for idx, row in df.iterrows():
            # Prepare payload
            customer_id = row.get('customer_id', f"CUST_{idx+1:04d}")
            
            # Extract features (exclude customer_id)
            features = {col: row[col] for col in df.columns 
                       if col != 'customer_id' and pd.notna(row[col])}
            
            payload = {
                "customer_id": customer_id,
                "features": features
            }
            
            # Make prediction
            result = make_prediction(payload)
            
            if "error" not in result:
                results.append({
                    "customer_id": customer_id,
                    "probability": result.get("probability"),
                    "predicted_class": result.get("predicted_class"),
                    "risk_category": result.get("risk_category"),
                    "risk_score": result.get("risk_score"),
                    "recommendation": result.get("recommendation"),
                    **features  # Include original features
                })
            else:
                results.append({
                    "customer_id": customer_id,
                    "error": result.get("error"),
                    **features
                })
            
            # Print progress
            if (idx + 1) % 10 == 0 or (idx + 1) == len(df):
                print(f"  Processed {idx + 1}/{len(df)}...", end="\r")
        
        print(f"\n✅ Processed {len(df)} records")
        
        # Create results DataFrame
        results_df = pd.DataFrame(results)
        
        # Save to file if specified
        if output_file:
            results_df.to_csv(output_file, index=False)
            print(f"📄 Results saved to: {output_file}")
        
        # Summary statistics
        successful = len([r for r in results if "error" not in r])
        print(f"\n📊 Batch Summary:")
        print(f"  Successful: {successful}/{len(df)}")
        print(f"  Failed: {len(df) - successful}")
        
        if successful > 0:
            print(f"  Avg probability: {results_df['probability'].mean():.4f}")
            risk_counts = results_df['risk_category'].value_counts()
            for risk, count in risk_counts.items():
                print(f"  {risk}: {count} customers ({count/len(results_df)*100:.1f}%)")
        
        return results_df
        
    except Exception as e:
        print(f"❌ Batch processing failed: {e}")
        return None

# %%
# Example: Create sample CSV for batch testing
sample_data = [
    {"customer_id": "BATCH_001", "Year_mean": 2024.0, "Month_mean": 6.0},
    {"customer_id": "BATCH_002", "Year_mean": 2023.0, "Month_mean": 3.0},
    {"customer_id": "BATCH_003", "Year_mean": 2022.0, "Month_mean": 11.0},
    {"customer_id": "BATCH_004", "Year_mean": 2024.0, "Month_mean": 8.0},
    {"customer_id": "BATCH_005", "Year_mean": 2023.0, "Month_mean": 5.0}
]

sample_df = pd.DataFrame(sample_data)
sample_csv_path = "../payloads/batch_sample.csv"
sample_df.to_csv(sample_csv_path, index=False)

print(f"✅ Created sample batch CSV: {sample_csv_path}")
print("\nSample data:")
print(sample_df.to_string())

# %%
# Uncomment to run batch prediction (requires API to be running)
# if api_healthy:
#     batch_results = batch_predict(sample_csv_path, "../reports/batch_results.csv")
#     print("\nBatch results preview:")
#     print(batch_results[['customer_id', 'probability', 'risk_category', 'recommendation']].to_string())

# %% [markdown]
# ## 12. API Client Class (for reuse in other projects)

# %%
class CreditRiskAPIClient:
    """Client class for Credit Risk Scoring API"""
    
    def __init__(self, base_url: str = "http://localhost:8001"):
        self.base_url = base_url
        self.health_endpoint = f"{base_url}/health"
        self.predict_endpoint = f"{base_url}/predict"
        self.model_info_endpoint = f"{base_url}/model-info"
        
    def check_health(self) -> Dict:
        """Check API health"""
        try:
            response = requests.get(self.health_endpoint, timeout=5)
            return {
                "healthy": response.status_code == 200,
                "status_code": response.status_code,
                "data": response.json() if response.status_code == 200 else None
            }
        except Exception as e:
            return {"healthy": False, "error": str(e)}
    
    def get_model_info(self) -> Dict:
        """Get model information"""
        try:
            response = requests.get(self.model_info_endpoint, timeout=5)
            if response.status_code == 200:
                return {"success": True, "data": response.json()}
            else:
                return {"success": False, "error": f"Status {response.status_code}"}
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    def predict(self, features: Dict, customer_id: str = None) -> Dict:
        """Make a prediction"""
        payload = {"features": features}
        if customer_id:
            payload["customer_id"] = customer_id
            
        try:
            response = requests.post(
                self.predict_endpoint,
                json=payload,
                headers={"Content-Type": "application/json"},
                timeout=10
            )
            
            if response.status_code == 200:
                return {"success": True, "data": response.json()}
            else:
                return {
                    "success": False,
                    "status_code": response.status_code,
                    "error": response.text
                }
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    def batch_predict(self, df: pd.DataFrame) -> pd.DataFrame:
        """Make predictions for a DataFrame of features"""
        results = []
        
        for idx, row in df.iterrows():
            customer_id = row.get('customer_id', f"cust_{idx}")
            features = {col: row[col] for col in df.columns if col != 'customer_id'}
            
            result = self.predict(features, customer_id)
            
            if result["success"]:
                data = result["data"]
                results.append({
                    "customer_id": customer_id,
                    "probability": data.get("probability"),
                    "predicted_class": data.get("predicted_class"),
                    "risk_category": data.get("risk_category"),
                    "risk_score": data.get("risk_score"),
                    "recommendation": data.get("recommendation"),
                    **features
                })
            else:
                results.append({
                    "customer_id": customer_id,
                    "error": result.get("error"),
                    **features
                })
        
        return pd.DataFrame(results)

# %%
# Example usage of the client class
print("Testing API Client Class...")
print("="*60)

# Create client
client = CreditRiskAPIClient(API_BASE_URL)

# Test health
health_result = client.check_health()
print(f"Health check: {'✅ Healthy' if health_result.get('healthy') else '❌ Unhealthy'}")

if health_result.get('healthy'):
    # Get model info
    model_info = client.get_model_info()
    if model_info.get('success'):
        print(f"Model: {model_info['data'].get('model_source', 'Unknown')}")
    
    # Make a prediction
    test_features = {"Year_mean": 2024.0, "Month_mean": 6.0}
    prediction = client.predict(test_features, "CLIENT_TEST_001")
    
    if prediction.get('success'):
        data = prediction['data']
        print(f"\nPrediction Result:")
        print(f"  Customer: {data.get('customer_id')}")
        print(f"  Probability: {data.get('probability'):.4f}")
        print(f"  Risk Category: {data.get('risk_category')}")
        print(f"  Recommendation: {data.get('recommendation')}")
    else:
        print(f"\n❌ Prediction failed: {prediction.get('error')}")

# %% [markdown]
# ## Summary
# 
# This notebook provides:
# 1. ✅ API connection testing
# 2. ✅ Sample payload generation
# 3. ✅ Response format validation
# 4. ✅ Performance testing
# 5. ✅ Error case testing
# 6. ✅ Integration test suite
# 7. ✅ Batch prediction example
# 8. ✅ Reusable API client class
# 
# To use this notebook:
# 1. Start your API: `docker-compose up -d`
# 2. Update `API_BASE_URL` if needed
# 3. Run the cells sequentially
# 4. Check the generated reports in the `reports/` directory

print("\n" + "="*60)
print("📚 API Inference Testing Complete!")
print("="*60)
print("\nNext steps:")
print("1. Check generated reports in ../reports/")
print("2. Review any failed tests")
print("3. Use the API client class in your applications")
print("4. Extend test cases as needed for your specific use case")